In [1]:
import numpy as np
from cvxopt import solvers, matrix, spmatrix, spdiag, sparse
import matplotlib.pyplot as plt
from softsvm import softsvm, fix_small_eigvals, error, get_random_sample
import softsvm
# unlimited np output
# np.set_printoptions(threshold=np.inf)
data = np.load(r'G:\My Drive\uni\Machine Learning intro\Introduction-To-Machine-Learning\ex2\ex2q4_data.npz')
trainX, testX = data['Xtrain'], data['Xtest']
trainY, testY = data['Ytrain'], data['Ytest']

In [2]:
def K(dot_res, k):
    """
    :param dot_res: dot product of two np.arrays
    :param k: the degree of the poly kernel
    :return: (1 + dot_res)^k
    """
    return np.power(float(1) + dot_res, k)

In [3]:
def get_gram_matrix(X, k):
    G = np.dot(X, X.T)
    # apply k to every item in G
    G = np.vectorize(lambda sample: K(sample, k))(G)
    return G
    

In [18]:
def softsvmpoly(l: float, k: int, trainX: np.array, trainy: np.array):
    """
    :param l: the parameter lambda of the soft SVM algorithm
    :param sigma: the bandwidth parameter sigma of the RBF kernel.
    :param trainX: numpy array of size (m, d) containing the training sample
    :param trainy: numpy array of size (m, 1) containing the labels of the training sample
    :return: numpy array of size (m, 1) which describes the coefficients found by the algorithm
    """
    m, d = trainX.shape
    
    G = get_gram_matrix(trainX, k)
    G = fix_small_eigvals(G)

    H = np.pad(float(2 * l) * G, [(0, m), (0, m)])
    H = fix_small_eigvals(H)

    A = np.block([[np.zeros((m, m)), np.identity(m)],
                  [G * trainy.reshape(-1, 1), np.identity(m)]])
    A = fix_small_eigvals(A)

    _G = min(np.linalg.eigvals(G))
    _H = min(np.linalg.eigvals(H))
    _A = min(np.linalg.eigvals(A))
    print("_G, _H, _A")
    print(_G, _H, _A)
    # A has negative eigenvalues!

    u = np.hstack((np.full(m, float(0)), np.full(m, 1/m)))

    v = np.hstack((np.zeros(m), np.ones(m)))
  
    z = solvers.qp(matrix(H), matrix(u), -matrix(A), -matrix(v))
    alphas = np.array(z["x"])[:m]
    return alphas

In [19]:
def simple_test():
    # load question 2 data
    data = np.load('EX2q2_mnist.npz')
    trainX = data['Xtrain']
    testX = data['Xtest']
    trainy = data['Ytrain']
    testy = data['Ytest']

    m = 100

    # Get a random m training examples from the training set
    indices = np.random.permutation(trainX.shape[0])
    _trainX = trainX[indices[:m]]
    _trainy = trainy[indices[:m]]

    # run the softsvmpoly algorithm
    w = softsvmpoly(10, 5, _trainX, _trainy)

    # tests to make sure the output is of the intended class and shape
    assert isinstance(
        w, np.ndarray), "The output of the function softsvmbf should be a numpy array"
    assert w.shape[0] == m and w.shape[
        1] == 1, f"The shape of the output should be ({m}, 1)"
simple_test()

_G, _H, _A
5.89698983965154e+31 2.220446049250313e-16 (-4.51955588371513e+17+0j)
     pcost       dcost       gap    pres   dres
 0: -1.0000e-02  1.0000e+00  2e+02  1e+00  7e+35
 1:  9.7039e-01 -1.9703e+00  3e+00  1e-02  7e+33
 2:  2.5568e-01 -1.2722e-02  3e-01  1e-04  7e+31
 3:  2.5817e-03 -1.0192e-04  3e-03  1e-06  7e+29
 4:  2.5817e-05 -1.0166e-06  3e-05  1e-08  7e+27
 5:  2.5817e-07 -1.0166e-08  3e-07  1e-10  7e+25
 6:  2.5817e-09 -1.0166e-10  3e-09  1e-12  7e+23
 7:  2.5817e-11 -1.0166e-12  3e-11  1e-14  7e+21
 8:  2.5817e-13 -1.0166e-14  3e-13  2e-15  7e+19
 9:  2.5817e-15 -1.0166e-16  3e-15  2e-15  7e+17
10:  2.5817e-17 -1.0166e-18  3e-17  2e-15  7e+15
11:  2.5817e-19 -1.0166e-20  3e-19  2e-15  7e+13
12:  2.5817e-21 -1.0166e-22  3e-21  2e-15  7e+11
13:  2.5817e-23 -1.0166e-24  3e-23  2e-15  7e+09
14:  2.5820e-25 -1.0167e-26  3e-25  1e-15  7e+07
15:  2.6056e-27 -1.0241e-28  3e-27  2e-15  7e+05
16:  4.6420e-29 -1.2808e-30  5e-29  1e-15  1e+04
17:  7.9877e-30  5.9062e-31  7e-30  1e

In [5]:
def cartesian_product(set_a: np.array, set_b: np.array):
    return [(ai, bi) for ai in set_a for bi in set_b]

In [6]:
def predict(alphas: np.array, k: int, testX: np.array):
    """
    :param alphas: numpy array of size (m, 1) containing the coefficients of the soft SVM algorithm
    :param testX: numpy array of size (m, d) containing the test sample
    :param k: int, the degree of the poly kernel
    :return: numpy array of size (m, 1) containing the predicted labels of the test sample
    """
    G = get_gram_matrix(testX, k)
    return np.sign(np.dot(G, alphas)) #TODO: shape?

In [7]:
def split_data(folds: int):
    X_chunks = np.array(np.split(trainX, folds))
    Y_chunks = np.array(np.split(trainY, folds))
    splitted = []
    shp = X_chunks.shape
    for i in range(folds):
        train, train_labales = X_chunks[i], Y_chunks[i]
        # current test set is the the entire train besides the current chunk used for training
        test = np.concatenate(np.delete(X_chunks, i, axis=0))
        test_labales = np.concatenate(np.delete(Y_chunks, i, axis=0))
        splitted.append({"train": train,
                        "train_labales": train_labales,
                        "test": test,
                        "test_labales": test_labales})
    return splitted

In [8]:
def poly_cross_validation(lambdas: np.array, ks, folds: int):
    """
    find pair (lambda, k) with the lowest validation error, and get classifier based on that pair
    :param lambdas: the lambda parameters to use
    :param ks: the k parameters to use in the kernel function
    :param folds: number of chunks we split the data to
    :return: prediction of test set of classifier trained on the entire train set using best lambda and k found
    """
    # poly softSVM
    errors = {(l, k): 0 for l, k in cartesian_product(lambdas, ks)}
    for fold in split_data(folds):
        for l, k in cartesian_product(lambdas, ks):
            alphas = softsvmpoly(l, k, fold["train"], fold["train_labales"])
            predicted = predict(alphas, k, fold["test"])
            # continuesly calculate the avg error
            errors[(l, k)] += error(fold["test_labales"], predicted) / folds
    
    # get the pair with lowest avg error
    best_lambda, best_k = min(error.items(), key=lambda x: x[1])[0]
    print(f"best_lambda={best_lambda}, best_k={best_k}")
    alphas = softsvmpoly(best_lambda, best_k, trainX, trainY)
    predict(alphas, best_k, testX)

In [9]:
def softsvm_cross_validation(lambdas, folds):
    errors = {l: 0 for l in lambdas}
    for fold in split_data(folds):
        for l in lambdas:
            w = softsvm(l, fold["train"], fold["train_labales"])
            errors[l] += error(fold["test_labales"], softsvm.predict(w, fold["test"])) / folds
    
    # get the l with lowest avg error
    best_lambda = min(error.items(), key=lambda x: x[1])[0]
    print(f"best_lambda={best_lambda}")
    w = softsvm(best_lambda, trainX, trainY)
    softsvm.predict(w, testX)

In [10]:
def cross_validation_error():
    """
    returns the error on test sample of
    clasiisfier from poly_cross_validation with 5 folds
    """
    lambdas = np.array([1, 10, 100])
    ks = np.array([2, 5, 8])

    # polynomial kernel
    poly_predicions = poly_cross_validation(lambdas, ks, 5)
    poly_error =  error(testY, poly_predicions)

    # linear softsvm
    soft_svm_predictions = poly_cross_validation(lambdas, ks, 5)
    soft_svm_error =  error(testY, soft_svm_predictions)
    print(f"soft_svm_error={soft_svm_error}")
    print(f"poly_error={poly_error}")


In [12]:
# def Q4_b():
#     cross_validation_error()
cross_validation_error()

ValueError: Rank(A) < p or Rank([P; A; G]) < n